In [1]:
#############################################################
# 1. Libraries

import pandas as pd
import numpy as np 
import os
import glob
from tqdm import tqdm
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
import matplotlib.pyplot as plt
import gc

import xgboost as xgb
import catboost

import pickle
from datetime import datetime
from pytorch_tabnet.tab_model import TabNetRegressor
from pytorch_tabnet.metrics import Metric
import torch

#############################################################

In [2]:
#############################################################
# 2. Paths & Global Variables

## 2.1 Paths

path = '../../../01_Data/'
path_sequences = path + '01_GeneratedSequences/'
path_spectograms_tiny = path + '02_GeneratedSpectograms_MelFeatures/'
path_spectograms_big = path + '03_GeneratedSpectograms_Scipy/'

path_models_mha = [f'../models/model_mha_{num_fold}' for num_fold in range(5)]
path_models_spectogram_tiny = [f'../models/model_cnn2d_Tiny_{num_fold}' for num_fold in range(5)]
path_models_spectogram_big = [f'../models/model_cnn2d_Big_{num_fold}' for num_fold in range(5)]

df_train_l2 = pd.read_csv('./df_all_train_l2.csv')
df_train = pd.read_csv(path + 'train.csv')
df_sample_submission = pd.read_csv(path + 'sample_submission.csv') 


unique_segments_id_train = set(df_train['segment_id'])
unique_segments_id_test = set(df_sample_submission['segment_id'])

dict_labels = {
    segment : df_train['time_to_eruption'][df_train['segment_id']==segment].values.flatten()[0]
    for segment in unique_segments_id_train
}

#############################################################

In [3]:
#############################################################
# 3. Global Functions

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        
#############################################################

In [4]:
#############################################################
# 5. Grid Search

grid_xgb = True
grid_catb = False

# 5.1 XGB 1h

train_columns = [col for col in df_train_l2.columns if col not in ['y_true', 'segment_id']]
X, y = df_train_l2[train_columns], df_train_l2['y_true']

# 90mins
if grid_xgb:
    print('Executing XGB Grid search.....')
    xgb_grid_params = {
            'n_estimators' : [100, 150, 200],
            'learning_rate': [0.01, 0.05, 0.1],
            'min_child_weight': [0.0, 0.2],
            'alpha': [0.5, 1.0, 2.0, 2.5, 3.0],
            'gamma': [0.5, 1.0, 2.0, 2.5, 3.0],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0], 
            'max_depth': [4, 5, 6]
    }

    xgb_model_grid = xgb.XGBRegressor(
        objective= 'reg:linear',
        nthread=1,
        seed=12
    )

    scorer = make_scorer(mean_absolute_error)

    grid_search = GridSearchCV(
        estimator=xgb_model_grid,
        param_grid=xgb_grid_params,
        scoring = 'neg_mean_absolute_error',
        n_jobs = 18,
        cv = 5,
        verbose=True
    )

    grid_search.fit(X, y)

    print('\n Score:')
    print(grid_search.best_score_)
    print('\n Best hyperparameters:')
    print(grid_search.best_params_)

    y_pred = grid_search.predict(X)

    X['xgb'] = y_pred
    X['y_true'] = y
    X = X.reset_index(drop=True)

    print(f"XGB: {np.mean(np.abs(X['y_true'] - X['xgb']))}")
    print(f"MHA: {np.mean(np.abs(X['y_true'] - X['mha_pred_mean']))}")
    print(f"CNN_TINY: {np.mean(np.abs(X['y_true'] - X['cnn_tiny_pred_mean']))}")
    print(f"CNN_BIG: {np.mean(np.abs(X['y_true'] - X['cnn_big_pred_mean']))}")
    print(f"MEAN MHA+TINY+BIG: {np.mean(np.abs(X['y_true'] - ((X['mha_pred_mean'] + X['cnn_tiny_pred_mean'] + X['cnn_big_pred_mean'])/3)))}")


# 5.2 Catboost 1h
# 200mins
if grid_catb:
    print('Executing CATBOOST Grid search.....')
    cat_grid_params = {
        'learning_rate' : [0.01, 0.025, 0.05, 0.1],
        'iterations': [300, 400, 500, 600, 700, 800],
        'depth': [3, 4, 5, 6],
        'l2_leaf_reg': [10, 15, 20, 30, 40],
        'bagging_temperature': [0.2, 0.3],
        'colsample_bylevel' : [0.7, 0.8, 0.9]
        # 'leaf_estimation_iterations': [10],
    }
    
    cat_model_grid = catboost.CatBoostRegressor(
        eval_metric='MAE',
        random_seed=12,
        early_stopping_rounds=100
    )
    
    scorer = make_scorer(mean_absolute_error)

    grid_search = GridSearchCV(
        estimator=cat_model_grid,
        param_grid=cat_grid_params,
        scoring = 'neg_mean_absolute_error',
        n_jobs = 16,
        cv = 5,
        verbose=True
    )
    
    grid_search.fit(X, y)

    print('\n Score:')
    print(grid_search.best_score_)
    print('\n Best hyperparameters:')
    print(grid_search.best_params_)
    
    y_pred = grid_search.predict(X)
    X['cat'] = y_pred
    X = X.reset_index(drop=True)

    print(f"CAT: {np.mean(np.abs(X['y_true'] - X['cat']))}")
    print(f"MHA: {np.mean(np.abs(X['y_true'] - X['mha_pred_mean']))}")
    print(f"CNN_TINY: {np.mean(np.abs(X['y_true'] - X['cnn_tiny_pred_mean']))}")
    print(f"CNN_BIG: {np.mean(np.abs(X['y_true'] - X['cnn_big_pred_mean']))}")
    print(f"MEAN MHA+TINY+BIG: {np.mean(np.abs(X['y_true'] - ((X['mha_pred_mean'] + X['cnn_tiny_pred_mean'] + X['cnn_big_pred_mean'])/3)))}")


#############################################################

Executing XGB Grid search.....
Fitting 5 folds for each of 12150 candidates, totalling 60750 fits


[Parallel(n_jobs=18)]: Using backend LokyBackend with 18 concurrent workers.
[Parallel(n_jobs=18)]: Done  14 tasks      | elapsed:    3.5s
[Parallel(n_jobs=18)]: Done 164 tasks      | elapsed:   14.8s
[Parallel(n_jobs=18)]: Done 414 tasks      | elapsed:   34.4s
[Parallel(n_jobs=18)]: Done 764 tasks      | elapsed:  1.1min
[Parallel(n_jobs=18)]: Done 1214 tasks      | elapsed:  1.7min
[Parallel(n_jobs=18)]: Done 1764 tasks      | elapsed:  2.4min
[Parallel(n_jobs=18)]: Done 2414 tasks      | elapsed:  3.3min
[Parallel(n_jobs=18)]: Done 3164 tasks      | elapsed:  4.3min
[Parallel(n_jobs=18)]: Done 4014 tasks      | elapsed:  5.5min
[Parallel(n_jobs=18)]: Done 4964 tasks      | elapsed:  6.9min
[Parallel(n_jobs=18)]: Done 6014 tasks      | elapsed:  8.5min
[Parallel(n_jobs=18)]: Done 7164 tasks      | elapsed: 10.2min
[Parallel(n_jobs=18)]: Done 8414 tasks      | elapsed: 12.2min
[Parallel(n_jobs=18)]: Done 9764 tasks      | elapsed: 14.4min
[Parallel(n_jobs=18)]: Done 11214 tasks      

[18:50:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[18:50:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.

 Score:
-1.3555247960408694

 Best hyperparameters:
{'alpha': 0.5, 'colsample_bytree': 0.9, 'gamma': 2.0, 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 0.0, 'n_estimators': 200, 'subsample': 0.8}
XGB: 1.026411279699399
MHA: 2.4882792013633526


C:\Users\Enric\anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: 'cnn_tiny_pred'

In [5]:
# {'alpha': 0.5, 'colsample_bytree': 0.8, 'gamma': 2.0, 'learning_rate': 0.05, 'max_depth': 4, 
#  'min_child_weight': 0.0, 'n_estimators': 150, 'subsample': 0.8}

# {'alpha': 0.5, 'colsample_bytree': 0.9, 'gamma': 2.0, 'learning_rate': 0.05, 'max_depth': 4, 
#  'min_child_weight': 0.0, 'n_estimators': 200, 'subsample': 0.8}

In [8]:
class TabnetMAE(Metric):
    def __init__(self):
        self._name = "mae"
        self._maximize = False

    def __call__(self, y_true, y_score):
        mae = mean_absolute_error(y_true, y_score[:, 1])
        return mae

In [9]:
#############################################################
# 7. Training L2 Stack Models

save = True
train_columns = [col for col in df_train_l2.columns if col not in ['y_true', 'segment_id']]
list_models_xgb, list_models_cat, list_models_tab = [], [], []

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=12)
list_segments_train = list(unique_segments_id_train)
dict_predictions = {'segment_id' : [], 'y_val_true' : [], 
                    'y_val_xgb_pred': [], 'y_val_cat_pred': [],  'y_val_tab_pred': []}

for num_fold, (train_index, val_index) in enumerate(kf.split(df_train_l2,
                                                     np.zeros(len(list_segments_train)))):
    
    X_train, y_train = df_train_l2[train_columns].loc[train_index], df_train_l2['y_true'].loc[train_index]
    X_val, y_val = df_train_l2[train_columns].loc[val_index], df_train_l2['y_true'].loc[val_index]
    
    print('=='*30)
    print(f'Num Fold: {num_fold + 1}')
    print(f'Train segments: {len(train_index)} Val segments: {len(val_index)}')
    print('=='*30)
    
    model_xgb = xgb.XGBRegressor(objective='reg:linear', booster='gbtree', tree_method='gpu_hist', #booster=gblinear
                                 learning_rate=0.05, max_depth=4, n_estimators=200, 
                                 alpha=0.5, gamma=2.0,
                                 subsample=0.8, colsample_bytree=0.9, min_child_weight=0.0,
                                 nthread=-1, 
                                 seed=12)
    
    model_cat = catboost.CatBoostRegressor(iterations=500, learning_rate=0.05, depth=5, 
                                 eval_metric='MAE', colsample_bylevel=0.7, bagging_temperature=0.2,
                                 early_stopping_rounds=100, l2_leaf_reg=10,
                                 random_seed=12)
    
    model_tabnet = TabNetRegressor()
    
    ######
    print('Training xgboost')
    model_xgb.fit(X_train, y_train, verbose=False) 
    y_val_xgb_pred = model_xgb.predict(X_val)
    print('Training catboost')
    model_cat.fit(X_train, y_train, verbose=False) 
    y_val_cat_pred = model_cat.predict(X_val)
    print('Training TabNet')
    model_tabnet.fit(
      X_train.values, 
      np.expand_dims(y_train.values, -1),
      eval_set=[(X_val.values, np.expand_dims(y_val.values, -1))],
      patience=50,
      max_epochs=250,
      eval_metric=[TabnetMAE],
      loss_fn=torch.nn.L1Loss()
    )
    y_val_tab_pred = model_tabnet.predict(X_val.values).squeeze()
    ##
    X_val['y_true'] = y_val
    X_val['tab'] = y_val_tab_pred
    X_val['xgb'] = y_val_xgb_pred
    X_val['cat'] = y_val_cat_pred
    X_val = X_val.reset_index(drop=True)
    ######

    print(f"XGB: {np.mean(np.abs(X_val['y_true'] - X_val['xgb']))}")
    print(f"CAT: {np.mean(np.abs(X_val['y_true'] - X_val['cat']))}")
    print(f"TAB: {np.mean(np.abs(X_val['y_true'] - X_val['tab']))}")
    print(f"MHA: {np.mean(np.abs(X_val['y_true'] - X_val['mha_pred_mean']))}")
    print(f"CNN_TINY: {np.mean(np.abs(X_val['y_true'] - X_val['cnn_tiny_pred_mean']))}")
    print(f"CNN_BIG: {np.mean(np.abs(X_val['y_true'] - X_val['cnn_big_pred_mean']))}")
    print(f"MEAN MHA+TINY+BIG: {np.mean(np.abs(X_val['y_true'] - ((X_val['mha_pred_mean']+X_val['cnn_tiny_pred_mean']+X_val['cnn_big_pred_mean'])/3)))}")
    print(f"MEAN XGB+CAT+TAB: {np.mean(np.abs(X_val['y_true'] - ((X_val['xgb'] + X_val['cat'] + X_val['tab'])/3)))}")
    
    dict_predictions['segment_id'].append(list(df_train_l2['segment_id'].loc[val_index]))
    dict_predictions['y_val_true'].append(list(df_train_l2['y_true'].loc[val_index]))
    dict_predictions['y_val_xgb_pred'].append(y_val_xgb_pred)
    dict_predictions['y_val_cat_pred'].append(y_val_cat_pred)
    dict_predictions['y_val_tab_pred'].append(y_val_tab_pred)
    
    list_models_xgb.append(model_xgb)
    list_models_cat.append(model_cat)
    list_models_tab.append(model_tabnet)
    print('**********'*10)

df_cv = pd.DataFrame({
    'segment_id' : np.concatenate(dict_predictions['segment_id']),
    'y_val_true' : np.concatenate(dict_predictions['y_val_true']),
    'y_val_xgb_pred' : np.concatenate(dict_predictions['y_val_xgb_pred']),
    'y_val_cat_pred' : np.concatenate(dict_predictions['y_val_cat_pred']),
    'y_val_tab_pred' : np.concatenate(dict_predictions['y_val_tab_pred'])
})

val_xgb_score = np.round(np.mean(np.abs(df_cv['y_val_true'] - df_cv['y_val_xgb_pred'])), 4)
val_cat_score = np.round(np.mean(np.abs(df_cv['y_val_true'] - df_cv['y_val_cat_pred'])), 4)
val_tab_score = np.round(np.mean(np.abs(df_cv['y_val_true'] - df_cv['y_val_tab_pred'])), 4)
val_ensem_score = np.round(np.mean(np.abs(df_cv['y_val_true'] - ((df_cv['y_val_xgb_pred'] + df_cv['y_val_cat_pred'] + df_cv['y_val_tab_pred'])/3))), 4)

print(f"XGB Score: {val_xgb_score}, CAT Score: {val_cat_score}, TAB Score: {val_tab_score}, ENSEM Score: {val_ensem_score}")
print('**********'*10)

if save:
    experiment_name = f"{val_ensem_score}_{datetime.today().strftime('%d-%m-%Y-%H-%M-%S')}"
    path = f'./StackedModels/{experiment_name}'
    if not os.path.exists(path):
        os.mkdir(path)
        for fold, model in enumerate(zip(list_models_xgb, list_models_cat, list_models_tab)):
            pickle.dump(model[0], open(f'{path}/xgb_{fold}.pickle', 'wb')) 
            pickle.dump(model[1], open(f'{path}/cat_{fold}.pickle', 'wb'))
            torch.save(model[2], f'{path}/tabnet_{fold}')

# v0.1 - Without conf
# ****************************************************************************************************
# XGB Score: 1.9784, CAT Score: 1.9637, ENSEM Score: 1.9602
# ****************************************************************************************************

# v0.2 - With conf
# ****************************************************************************************************
# XGB Score: 1.9357, CAT Score: 1.904, ENSEM Score: 1.9069
# ****************************************************************************************************

# v0.3 - With conf & wavenet with test stats
# ****************************************************************************************************
# XGB Score: 1.9537, CAT Score: 1.9416, ENSEM Score: 1.9339
# ****************************************************************************************************

# v0.4 - With stft
# ****************************************************************************************************
# XGB Score: 1.9531, CAT Score: 1.9438, ENSEM Score: 1.9325
# ****************************************************************************************************

# v0.5 - With lgbm stft
# ****************************************************************************************************
# XGB Score: 1.7674, CAT Score: 1.7744, ENSEM Score: 1.7552
# ****************************************************************************************************

# v0.6 - Add tabnet
# ****************************************************************************************************
# XGB Score: 1.7766, CAT Score: 1.7809, TAB Score: 1.744, ENSEM Score: 1.7143
# ****************************************************************************************************

# v0.7 - Updated Models with new tta
# ****************************************************************************************************
# XGB Score: 1.5708, CAT Score: 1.5869, TAB Score: 1.6282, ENSEM Score: 1.5391
# ****************************************************************************************************

# v0.8 - Updated Models with new tta + predictions tabular
# ****************************************************************************************************
# XGB Score: 1.3259, CAT Score: 1.3528, TAB Score: 1.3179, ENSEM Score: 1.2849
# ****************************************************************************************************

# v0.9 - Updated Models with new tta + predictions tabular + hipertune xgb
# ****************************************************************************************************
# XGB Score: 1.329, CAT Score: 1.3528, TAB Score: 1.3179, ENSEM Score: 1.2851
# ****************************************************************************************************

Num Fold: 1
Train segments: 3544 Val segments: 887
Device used : cuda
Training xgboost
[19:17:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
Training catboost
Training TabNet
epoch 0  | loss: 23.59666| val_0_mae: 25.82856|  0:00:02s
epoch 1  | loss: 22.58278| val_0_mae: 23.40626|  0:00:02s
epoch 2  | loss: 21.94629| val_0_mae: 20.02263|  0:00:02s
epoch 3  | loss: 21.25004| val_0_mae: 12.88641|  0:00:03s
epoch 4  | loss: 20.31042| val_0_mae: 12.62921|  0:00:03s
epoch 5  | loss: 19.13432| val_0_mae: 9.55822 |  0:00:03s
epoch 6  | loss: 17.65253| val_0_mae: 8.84495 |  0:00:03s
epoch 7  | loss: 15.90367| val_0_mae: 9.23727 |  0:00:03s
epoch 8  | loss: 13.92524| val_0_mae: 9.3713  |  0:00:04s
epo

epoch 131| loss: 1.5621  | val_0_mae: 1.61103 |  0:00:28s
epoch 132| loss: 1.48142 | val_0_mae: 1.53228 |  0:00:28s
epoch 133| loss: 1.49032 | val_0_mae: 1.60905 |  0:00:28s
epoch 134| loss: 1.49219 | val_0_mae: 1.42555 |  0:00:28s
epoch 135| loss: 1.5227  | val_0_mae: 1.49288 |  0:00:28s
epoch 136| loss: 1.54507 | val_0_mae: 1.54579 |  0:00:29s
epoch 137| loss: 1.36915 | val_0_mae: 1.37485 |  0:00:29s
epoch 138| loss: 1.46379 | val_0_mae: 1.38748 |  0:00:29s
epoch 139| loss: 1.4183  | val_0_mae: 1.67885 |  0:00:29s
epoch 140| loss: 1.49419 | val_0_mae: 1.55611 |  0:00:29s
epoch 141| loss: 1.59483 | val_0_mae: 1.40605 |  0:00:30s
epoch 142| loss: 1.52692 | val_0_mae: 1.44879 |  0:00:30s
epoch 143| loss: 1.49259 | val_0_mae: 1.44275 |  0:00:30s
epoch 144| loss: 1.4878  | val_0_mae: 1.46143 |  0:00:30s
epoch 145| loss: 1.42909 | val_0_mae: 1.43272 |  0:00:30s
epoch 146| loss: 1.4193  | val_0_mae: 1.37177 |  0:00:31s
epoch 147| loss: 1.47371 | val_0_mae: 1.40955 |  0:00:31s
epoch 148| los

epoch 4  | loss: 20.58188| val_0_mae: 17.02153|  0:00:00s
epoch 5  | loss: 19.46118| val_0_mae: 14.42634|  0:00:01s
epoch 6  | loss: 18.02976| val_0_mae: 10.84923|  0:00:01s
epoch 7  | loss: 16.2714 | val_0_mae: 7.94944 |  0:00:01s
epoch 8  | loss: 14.22001| val_0_mae: 8.08145 |  0:00:01s
epoch 9  | loss: 11.81805| val_0_mae: 7.76848 |  0:00:01s
epoch 10 | loss: 9.10733 | val_0_mae: 10.47688|  0:00:02s
epoch 11 | loss: 6.37891 | val_0_mae: 16.40004|  0:00:02s
epoch 12 | loss: 3.9409  | val_0_mae: 24.95805|  0:00:02s
epoch 13 | loss: 3.27744 | val_0_mae: 25.24245|  0:00:02s
epoch 14 | loss: 3.17753 | val_0_mae: 17.56455|  0:00:02s
epoch 15 | loss: 2.59362 | val_0_mae: 13.22283|  0:00:03s
epoch 16 | loss: 2.41929 | val_0_mae: 13.59539|  0:00:03s
epoch 17 | loss: 2.18275 | val_0_mae: 15.37704|  0:00:03s
epoch 18 | loss: 2.26018 | val_0_mae: 14.21726|  0:00:03s
epoch 19 | loss: 2.05895 | val_0_mae: 13.5312 |  0:00:03s
epoch 20 | loss: 2.15603 | val_0_mae: 13.74895|  0:00:04s
epoch 21 | los

epoch 147| loss: 1.4405  | val_0_mae: 1.39349 |  0:00:28s
epoch 148| loss: 1.38993 | val_0_mae: 1.34328 |  0:00:28s
epoch 149| loss: 1.4466  | val_0_mae: 1.39517 |  0:00:28s
epoch 150| loss: 1.39404 | val_0_mae: 1.51016 |  0:00:29s
epoch 151| loss: 1.5756  | val_0_mae: 1.40897 |  0:00:29s
epoch 152| loss: 1.43    | val_0_mae: 1.4816  |  0:00:29s
epoch 153| loss: 1.37897 | val_0_mae: 1.451   |  0:00:29s
epoch 154| loss: 1.44372 | val_0_mae: 1.49126 |  0:00:29s
epoch 155| loss: 1.43164 | val_0_mae: 1.4026  |  0:00:30s
epoch 156| loss: 1.49747 | val_0_mae: 1.50221 |  0:00:30s
epoch 157| loss: 1.53596 | val_0_mae: 1.46493 |  0:00:30s
epoch 158| loss: 1.43903 | val_0_mae: 1.54469 |  0:00:30s
epoch 159| loss: 1.77119 | val_0_mae: 1.44838 |  0:00:30s
epoch 160| loss: 1.41335 | val_0_mae: 1.53373 |  0:00:31s
epoch 161| loss: 1.555   | val_0_mae: 1.45134 |  0:00:31s
epoch 162| loss: 1.46989 | val_0_mae: 1.39777 |  0:00:31s
epoch 163| loss: 1.53295 | val_0_mae: 1.56494 |  0:00:31s
epoch 164| los

epoch 47 | loss: 1.72541 | val_0_mae: 3.52827 |  0:00:09s
epoch 48 | loss: 1.84266 | val_0_mae: 3.89097 |  0:00:09s
epoch 49 | loss: 1.80922 | val_0_mae: 3.14945 |  0:00:10s
epoch 50 | loss: 1.83096 | val_0_mae: 3.44316 |  0:00:10s
epoch 51 | loss: 1.7106  | val_0_mae: 2.9579  |  0:00:10s
epoch 52 | loss: 1.75237 | val_0_mae: 3.60106 |  0:00:10s
epoch 53 | loss: 1.69632 | val_0_mae: 2.89464 |  0:00:10s
epoch 54 | loss: 1.73111 | val_0_mae: 2.75798 |  0:00:10s
epoch 55 | loss: 1.73566 | val_0_mae: 3.47462 |  0:00:11s
epoch 56 | loss: 1.69017 | val_0_mae: 2.91556 |  0:00:11s
epoch 57 | loss: 1.88517 | val_0_mae: 4.00029 |  0:00:11s
epoch 58 | loss: 1.81293 | val_0_mae: 2.38161 |  0:00:11s
epoch 59 | loss: 1.84698 | val_0_mae: 2.54127 |  0:00:11s
epoch 60 | loss: 1.69498 | val_0_mae: 2.67789 |  0:00:12s
epoch 61 | loss: 1.63405 | val_0_mae: 2.15853 |  0:00:12s
epoch 62 | loss: 1.77243 | val_0_mae: 2.77614 |  0:00:12s
epoch 63 | loss: 1.72795 | val_0_mae: 1.83431 |  0:00:12s
epoch 64 | los

epoch 190| loss: 1.51013 | val_0_mae: 1.42183 |  0:00:37s
epoch 191| loss: 1.61511 | val_0_mae: 1.39986 |  0:00:38s
epoch 192| loss: 1.60811 | val_0_mae: 1.81797 |  0:00:38s
epoch 193| loss: 1.46983 | val_0_mae: 2.00723 |  0:00:38s
epoch 194| loss: 1.59961 | val_0_mae: 1.6452  |  0:00:38s
epoch 195| loss: 1.56231 | val_0_mae: 1.52899 |  0:00:38s
epoch 196| loss: 1.49307 | val_0_mae: 1.44543 |  0:00:39s
epoch 197| loss: 1.45945 | val_0_mae: 1.33044 |  0:00:39s
epoch 198| loss: 1.42186 | val_0_mae: 1.41956 |  0:00:39s
epoch 199| loss: 1.42815 | val_0_mae: 1.36009 |  0:00:39s
epoch 200| loss: 1.41652 | val_0_mae: 1.34752 |  0:00:39s
epoch 201| loss: 1.48263 | val_0_mae: 1.34228 |  0:00:40s
epoch 202| loss: 1.44308 | val_0_mae: 1.36891 |  0:00:40s
epoch 203| loss: 1.39566 | val_0_mae: 1.4718  |  0:00:40s
epoch 204| loss: 1.49168 | val_0_mae: 1.38717 |  0:00:40s
epoch 205| loss: 1.36371 | val_0_mae: 1.36308 |  0:00:40s
epoch 206| loss: 1.42768 | val_0_mae: 1.40766 |  0:00:41s
epoch 207| los

epoch 65 | loss: 1.66181 | val_0_mae: 1.74484 |  0:00:13s
epoch 66 | loss: 1.63035 | val_0_mae: 2.10875 |  0:00:13s
epoch 67 | loss: 1.63717 | val_0_mae: 2.25269 |  0:00:13s
epoch 68 | loss: 1.56326 | val_0_mae: 2.50211 |  0:00:14s
epoch 69 | loss: 1.69684 | val_0_mae: 2.24847 |  0:00:14s
epoch 70 | loss: 1.58366 | val_0_mae: 2.07946 |  0:00:14s
epoch 71 | loss: 1.79062 | val_0_mae: 2.27482 |  0:00:14s
epoch 72 | loss: 1.54613 | val_0_mae: 2.07494 |  0:00:14s
epoch 73 | loss: 1.69961 | val_0_mae: 2.14863 |  0:00:14s
epoch 74 | loss: 1.67966 | val_0_mae: 2.13456 |  0:00:15s
epoch 75 | loss: 1.69212 | val_0_mae: 1.42296 |  0:00:15s
epoch 76 | loss: 1.76024 | val_0_mae: 1.59236 |  0:00:15s
epoch 77 | loss: 1.49638 | val_0_mae: 1.84424 |  0:00:15s
epoch 78 | loss: 1.4975  | val_0_mae: 1.5413  |  0:00:15s
epoch 79 | loss: 1.68893 | val_0_mae: 1.54238 |  0:00:16s
epoch 80 | loss: 1.49459 | val_0_mae: 2.04976 |  0:00:16s
epoch 81 | loss: 1.51149 | val_0_mae: 1.57709 |  0:00:16s
epoch 82 | los

epoch 208| loss: 1.54596 | val_0_mae: 1.41662 |  0:00:41s
epoch 209| loss: 1.39509 | val_0_mae: 1.52095 |  0:00:41s
epoch 210| loss: 1.49003 | val_0_mae: 1.37913 |  0:00:41s
epoch 211| loss: 1.39831 | val_0_mae: 1.34951 |  0:00:42s
epoch 212| loss: 1.34879 | val_0_mae: 1.34539 |  0:00:42s
epoch 213| loss: 1.28637 | val_0_mae: 1.44044 |  0:00:42s
epoch 214| loss: 1.30908 | val_0_mae: 1.34438 |  0:00:42s
epoch 215| loss: 1.34929 | val_0_mae: 1.32617 |  0:00:42s
epoch 216| loss: 1.32801 | val_0_mae: 1.32728 |  0:00:43s
epoch 217| loss: 1.27159 | val_0_mae: 1.27399 |  0:00:43s
epoch 218| loss: 1.36814 | val_0_mae: 1.31212 |  0:00:43s
epoch 219| loss: 1.36163 | val_0_mae: 1.29237 |  0:00:43s
epoch 220| loss: 1.29115 | val_0_mae: 1.2627  |  0:00:43s
epoch 221| loss: 1.33392 | val_0_mae: 1.28326 |  0:00:44s
epoch 222| loss: 1.31318 | val_0_mae: 1.29855 |  0:00:44s
epoch 223| loss: 1.38136 | val_0_mae: 1.30239 |  0:00:44s
epoch 224| loss: 1.45051 | val_0_mae: 1.3703  |  0:00:44s
epoch 225| los

epoch 82 | loss: 1.39308 | val_0_mae: 1.64761 |  0:00:16s
epoch 83 | loss: 1.57265 | val_0_mae: 1.80698 |  0:00:16s
epoch 84 | loss: 1.52312 | val_0_mae: 1.80363 |  0:00:16s
epoch 85 | loss: 1.5166  | val_0_mae: 2.07074 |  0:00:17s
epoch 86 | loss: 1.50842 | val_0_mae: 1.82598 |  0:00:17s
epoch 87 | loss: 1.42281 | val_0_mae: 1.78481 |  0:00:17s
epoch 88 | loss: 1.49789 | val_0_mae: 1.95925 |  0:00:17s
epoch 89 | loss: 1.51717 | val_0_mae: 1.67806 |  0:00:17s
epoch 90 | loss: 1.42859 | val_0_mae: 1.50202 |  0:00:18s
epoch 91 | loss: 1.53976 | val_0_mae: 1.77604 |  0:00:18s
epoch 92 | loss: 1.41454 | val_0_mae: 1.60058 |  0:00:18s
epoch 93 | loss: 1.44079 | val_0_mae: 1.72213 |  0:00:18s
epoch 94 | loss: 1.50996 | val_0_mae: 1.50526 |  0:00:18s
epoch 95 | loss: 1.51539 | val_0_mae: 1.632   |  0:00:19s
epoch 96 | loss: 1.67967 | val_0_mae: 1.52282 |  0:00:19s
epoch 97 | loss: 1.5476  | val_0_mae: 1.45723 |  0:00:19s
epoch 98 | loss: 1.58044 | val_0_mae: 1.5312  |  0:00:19s
epoch 99 | los

epoch 224| loss: 1.35147 | val_0_mae: 1.48563 |  0:00:44s
epoch 225| loss: 1.40088 | val_0_mae: 1.66367 |  0:00:44s
epoch 226| loss: 1.63903 | val_0_mae: 1.90657 |  0:00:44s
epoch 227| loss: 1.43629 | val_0_mae: 1.66951 |  0:00:45s
epoch 228| loss: 1.44589 | val_0_mae: 1.48735 |  0:00:45s
epoch 229| loss: 1.41744 | val_0_mae: 1.44452 |  0:00:45s
epoch 230| loss: 1.44204 | val_0_mae: 1.4173  |  0:00:45s
epoch 231| loss: 1.42981 | val_0_mae: 1.41251 |  0:00:45s
epoch 232| loss: 1.35106 | val_0_mae: 1.59074 |  0:00:46s
epoch 233| loss: 1.35329 | val_0_mae: 1.57337 |  0:00:46s
epoch 234| loss: 1.38538 | val_0_mae: 1.51191 |  0:00:46s
epoch 235| loss: 1.37456 | val_0_mae: 1.45224 |  0:00:46s
epoch 236| loss: 1.38847 | val_0_mae: 1.45116 |  0:00:46s
epoch 237| loss: 1.36349 | val_0_mae: 1.45407 |  0:00:47s
epoch 238| loss: 1.34568 | val_0_mae: 1.50101 |  0:00:47s
epoch 239| loss: 1.31456 | val_0_mae: 1.48201 |  0:00:47s
epoch 240| loss: 1.51422 | val_0_mae: 1.54792 |  0:00:47s
epoch 241| los